# ⚙️ Analyse des Opportunités d'Automatisation

## Objectif
Identifier et prioriser les processus opérationnels à automatiser dans les cliniques.

---

## 📚 1. Imports et Configuration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

pd.set_option('display.max_columns', None)
print("✅ Imports réussis")

## 📂 2. Chargement des Données

In [ ]:
df = pd.read_csv('../../donnees_cliniques_nettoyees.csv')
print(f"📊 {df.shape[0]} cliniques, {df.shape[1]} colonnes")
df.head(3)

## ⏱️ 3. Analyse des Processus Chronophages

In [ ]:
# Colonnes des tâches chronophages
tache_columns = {
    'time_consuming_appointments': 'Prise de rendez-vous',
    'time_consuming_patient_records': 'Gestion dossiers patients',
    'time_consuming_billing_payment': 'Facturation et paiements',
    'time_consuming_inventory_orders': 'Gestion stocks/commandes'
}

def convert_to_numeric(val):
    if pd.isna(val):
        return 0
    val_str = str(val).lower().strip()
    return 1 if val_str in ['1', '1.0', 'oui', 'yes'] else 0

tache_data = []
for col, label in tache_columns.items():
    if col in df.columns:
        count = df[col].apply(convert_to_numeric).sum()
        pct = (count / len(df)) * 100
        tache_data.append({
            'Processus': label,
            'Nb Cliniques': int(count),
            'Pourcentage': pct
        })

df_taches = pd.DataFrame(tache_data).sort_values('Pourcentage', ascending=False)
print("\n⏱️ PROCESSUS LES PLUS CHRONOPHAGES")
print("="*60)
print(df_taches.to_string(index=False))
df_taches

In [ ]:
# Visualisation
fig = px.bar(df_taches, x='Pourcentage', y='Processus', orientation='h',
             title='⏱️ Processus les Plus Chronophages',
             labels={'Pourcentage': 'Pourcentage de cliniques (%)', 'Processus': ''},
             color='Pourcentage', color_continuous_scale='Reds',
             text='Pourcentage')
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(height=450, showlegend=False, xaxis_range=[0, 100])
fig.write_html('reports/processus_chronophages.html')
fig.show()
print("✅ Graphique: reports/processus_chronophages.html")

## 🤖 4. Demande d'Automatisation

In [ ]:
# Colonnes automatisation
auto_columns = {
    'want_automation_appointments': 'Prise de rendez-vous',
    'want_automation_patient_records': 'Gestion dossiers patients',
    'want_automation_billing': 'Facturation',
    'want_automation_data_analysis': 'Analyse de données',
    'want_automation_inventory': 'Gestion des stocks'
}

auto_data = []
for col, label in auto_columns.items():
    if col in df.columns:
        count = df[col].apply(convert_to_numeric).sum()
        pct = (count / len(df)) * 100
        auto_data.append({
            'Processus': label,
            'Demande': int(count),
            'Pourcentage': pct
        })

df_auto = pd.DataFrame(auto_data).sort_values('Pourcentage', ascending=False)

# Ajouter priorité
def get_priority(pct):
    if pct >= 85: return '🔴 TRÈS HAUTE'
    elif pct >= 70: return '🟡 HAUTE'
    elif pct >= 50: return '🟢 MOYENNE'
    else: return '⚪ FAIBLE'

df_auto['Priorité'] = df_auto['Pourcentage'].apply(get_priority)

print("\n🤖 DEMANDE D'AUTOMATISATION PAR PROCESSUS")
print("="*70)
print(df_auto.to_string(index=False))
df_auto

In [ ]:
# Visualisation demande
fig = px.bar(df_auto, x='Pourcentage', y='Processus', orientation='h',
             title='🤖 Demande d\'Automatisation par Processus',
             labels={'Pourcentage': 'Pourcentage de cliniques (%)', 'Processus': ''},
             color='Pourcentage', color_continuous_scale='Greens',
             text='Pourcentage')
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(height=500, showlegend=False, xaxis_range=[0, 100])
fig.write_html('reports/demande_automatisation.html')
fig.show()
print("✅ Graphique: reports/demande_automatisation.html")

## 📊 5. Matrice Effort vs Impact

In [ ]:
# Créer matrice effort/impact
processes_matrix = [
    {'Processus': 'Prise de RDV', 'Impact': 95, 'Effort': 30, 'ROI_mois': 4, 'Gain_h_semaine': 8},
    {'Processus': 'Facturation', 'Impact': 85, 'Effort': 25, 'ROI_mois': 3, 'Gain_h_semaine': 4},
    {'Processus': 'Dossiers Patients', 'Impact': 90, 'Effort': 70, 'ROI_mois': 9, 'Gain_h_semaine': 12},
    {'Processus': 'Analyse Données', 'Impact': 80, 'Effort': 65, 'ROI_mois': 8, 'Gain_h_semaine': 5},
    {'Processus': 'Gestion Stocks', 'Impact': 70, 'Effort': 60, 'ROI_mois': 10, 'Gain_h_semaine': 3},
    {'Processus': 'Rappels SMS', 'Impact': 75, 'Effort': 15, 'ROI_mois': 2, 'Gain_h_semaine': 2}
]

df_matrix = pd.DataFrame(processes_matrix)

# Catégoriser
def categorize_matrix(row):
    if row['Effort'] < 40 and row['Impact'] >= 80:
        return '🟢 Quick Win'
    elif row['Effort'] >= 60 and row['Impact'] >= 80:
        return '🟡 Stratégique'
    elif row['Effort'] < 40 and row['Impact'] < 80:
        return '🔵 Gain Marginal'
    else:
        return '🔴 À Reporter'

df_matrix['Catégorie'] = df_matrix.apply(categorize_matrix, axis=1)

print("\n📊 MATRICE EFFORT VS IMPACT")
print("="*80)
print(df_matrix.to_string(index=False))

In [ ]:
# Visualisation matrice
color_map = {
    '🟢 Quick Win': '#2ecc71',
    '🟡 Stratégique': '#f39c12',
    '🔵 Gain Marginal': '#3498db',
    '🔴 À Reporter': '#e74c3c'
}

fig = px.scatter(df_matrix, x='Effort', y='Impact', 
                 size='Gain_h_semaine', color='Catégorie',
                 text='Processus', size_max=60,
                 title='📊 Matrice Effort vs Impact',
                 labels={'Effort': 'Effort d\'implémentation', 'Impact': 'Impact sur l\'efficacité'},
                 color_discrete_map=color_map)

fig.update_traces(textposition='top center')
fig.add_shape(type="line", x0=50, y0=0, x1=50, y1=100, line=dict(color="gray", dash="dash"))
fig.add_shape(type="line", x0=0, y0=75, x1=100, y1=75, line=dict(color="gray", dash="dash"))
fig.update_layout(height=600, xaxis_range=[0, 100], yaxis_range=[0, 100])
fig.write_html('reports/matrice_effort_impact.html')
fig.show()
print("✅ Graphique: reports/matrice_effort_impact.html")

## 🎯 6. Priorisation et Roadmap

In [ ]:
# Trier par priorité (Quick Wins d'abord, puis Stratégiques)
df_priority = df_matrix.sort_values(['Catégorie', 'Impact'], ascending=[True, False])

print("\n🎯 ROADMAP PRIORISÉE")
print("="*80)

for cat in ['�� Quick Win', '🟡 Stratégique', '🔵 Gain Marginal', '🔴 À Reporter']:
    processes = df_priority[df_priority['Catégorie'] == cat]
    if len(processes) > 0:
        print(f"\n{cat}:")
        for _, p in processes.iterrows():
            print(f"  • {p['Processus']} - Impact: {p['Impact']}% | Effort: {p['Effort']}% | ROI: {p['ROI_mois']}mois | Gain: {p['Gain_h_semaine']}h/sem")

## 💰 7. Calcul des Gains Potentiels

In [ ]:
# Calculer gains par processus
print("\n💰 GAINS ESTIMÉS PAR PROCESSUS")
print("="*80)

for _, row in df_matrix.iterrows():
    processus = row['Processus']
    gain_h = row['Gain_h_semaine']
    nb_cliniques = int(df_auto[df_auto['Processus'].str.contains(processus.split()[0], case=False, na=False)]['Demande'].values[0] if len(df_auto[df_auto['Processus'].str.contains(processus.split()[0], case=False, na=False)]) > 0 else 20)
    
    gain_mensuel_h = gain_h * 4 * nb_cliniques
    cout_horaire = 2500  # FCFA
    economie_mensuelle = gain_mensuel_h * cout_horaire
    economie_annuelle = economie_mensuelle * 12
    
    print(f"\n{processus}:")
    print(f"  • Gain/clinique: {gain_h}h/semaine ({gain_h*4}h/mois)")
    print(f"  • Cliniques cibles: {nb_cliniques}")
    print(f"  • Gain total: {gain_mensuel_h}h/mois")
    print(f"  • Économie mensuelle: {economie_mensuelle:,.0f} FCFA")
    print(f"  • Économie annuelle: {economie_annuelle:,.0f} FCFA")

# Total
total_gain_h = df_matrix['Gain_h_semaine'].sum() * 4 * 20  # Moyenne 20 cliniques
total_economie = total_gain_h * cout_horaire * 12
print(f"\n{'='*80}")
print(f"POTENTIEL TOTAL: {total_gain_h}h/mois | {total_economie:,.0f} FCFA/an")

## 📊 8. Export des Résultats

In [ ]:
# Exporter
df_taches.to_csv('data/processus_chronophages.csv', index=False)
df_auto.to_csv('data/demande_automatisation.csv', index=False)
df_matrix.to_csv('data/matrice_effort_impact.csv', index=False)

print("✅ Données exportées:")
print("  - processus_chronophages.csv")
print("  - demande_automatisation.csv")
print("  - matrice_effort_impact.csv")

## 📝 9. Conclusions et Recommandations

### 🎯 Conclusions Clés

1. **Quick Wins Identifiés**
   - Prise de RDV: 95% d'impact, 30% d'effort → À déployer immédiatement
   - Facturation: 85% d'impact, 25% d'effort → Déploiement rapide
   - ROI de 2-4 mois pour ces solutions

2. **Projets Stratégiques**
   - Dossiers Patients: 90% d'impact, 70% d'effort → Phase 2
   - Analyse de Données: 80% d'impact, 65% d'effort → Phase 2
   - ROI de 8-10 mois mais gains majeurs

3. **Gains Potentiels**
   - 20-35 heures/semaine libérées par clinique
   - Équivalent à 1 ETP récupéré
   - Économies de 3-5 millions FCFA/an par clinique

### 🚀 Recommandations

#### Phase 1 (Mois 1-3): Quick Wins
1. ✅ Déployer module RDV en ligne (15-20 cliniques)
2. ✅ Déployer module Facturation (15-20 cliniques)
3. ✅ Mesurer les gains et créer des cas d'usage

#### Phase 2 (Mois 4-9): Stratégique
1. ✅ Développer et déployer DPE (10-15 cliniques)
2. ✅ Intégrer l'analyse de données (8-12 cliniques)
3. ✅ Optimiser les workflows

#### Phase 3 (Mois 10-18): Optimisation
1. ✅ Gestion intelligente des stocks
2. ✅ Intégration complète des systèmes
3. ✅ IA prédictive et automatisation avancée

---

**L'automatisation n'est plus une option, c'est une nécessité ! 🚀**